
# ICRA Tutorial 4, Contradictory Prior Data 2D

## Overview

- This example shows how prior (possibly contradictor) data can be incorporated into a localization and mapping solution.
- The value of having prior data for localization and mapping can be enormous, yet the risk of inducing unwanted errors due to contradictory data in the final problem are equally large -- this tutorial will show NavAbility's method of leveraging contradictory prior data as well as detecting discrepancies between "as-designed" and "as-built" artifacts.
- This tutorial will show how to use non-Gaussian inference to resolve the duality problem between measurement errors and contradictory data models by using the multi-hypothesis features in a factor graph solution.
- This tutorial is in 2D on a XY-plane, assuming a top down view.
- We assume a human operated surveyor measurement station is used for data collection to known markers in a construction environment.
- The data used to construct this tutorial is used to generate a factor graph which is solved and the results analyzed.
- The data is based on multiple measurement station locations to known makers in the environment.
- The measurement locations are not known in advance, but the relative translation between the measurement locations is known.
- The known marker locations are known only from the as-designed prior inforation -- for example a CAD model or floor plan of a construction.
- To simplify the tutorial, the measurements are assumed to be purely Gaussian and non-Guassian behavior is introduced only through the multi-hypothesis mechanism.
- This example shows one of four possible mechanisms by which non-Gaussian behavior can be introduced to a factor graph system.
- The multi-hypothesis mechanism used in this tutorial assumes the user will introduce discrete uncertainty into the estimation problem at the factor graph level.
  - This is difference from ICRA Tutorial 2 where discrete uncertainty about ambiguous data is marginalized into the measurement before a factor is created.
- Other mechanisms shown in other examples can readily be added to this tutorial for a more intricate system.


## As-Designed vs. As-Built

SIMILAR/SIMPLER VERSION OF CONSTRUCTION EXAMPLE FOR RAL PAPER

In [ ]:
using RoME